<a href="https://colab.research.google.com/github/Koperniko1/SQL-New-York-Collisions/blob/main/SQL_New_York_Collisions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importar librerias**

In [1]:
!pip install plotly

In [2]:
import pandas as pd
import plotly.express as px
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()

In [3]:
# Conectar al proyecto de BigQuery
project_id = 'nuevo-proyecto-452915'  # ID del proyecto de Google Cloud
client = bigquery.Client(project=project_id)

# Función para ejecutar consultas y devolver el resultado como DataFrame
def ejecutar_consulta(query):
    return client.query(query).to_dataframe()


## **Analisis exploratorio**

### **Obtener Información de la Tabla**

In [4]:
# Consulta para obtener la información de las columnas de la tabla
query = """
SELECT
    column_name,
    data_type
FROM
    `bigquery-public-data.new_york_mv_collisions.INFORMATION_SCHEMA.COLUMNS`
WHERE
    table_name = 'nypd_mv_collisions'
"""

# Ejecutar la consulta y obtener los datos en un DataFrame
df_info = ejecutar_consulta(query)

# Mostrar la información de las columnas
df_info


,column_name,data_type
0,borough,STRING
1,contributing_factor_vehicle_1,STRING
2,contributing_factor_vehicle_2,STRING
3,contributing_factor_vehicle_3,STRING
4,contributing_factor_vehicle_4,STRING
5,contributing_factor_vehicle_5,STRING
6,cross_street_name,STRING
7,timestamp,DATETIME
8,latitude,FLOAT64
9,longitude,FLOAT64


In [5]:
# Consulta para obtener el rango de fechas
query = """
SELECT
    MIN(timestamp) AS fecha_minima,
    MAX(timestamp) AS fecha_maxima
FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions`

"""

# Ejecutar la consulta y obtener los datos en un DataFrame
df_time = ejecutar_consulta(query)

# Mostrar el resultado
df_time


,fecha_minima,fecha_maxima
0,2012-07-01 00:05:00,2025-03-04 23:22:00


### **Análisis Temporal: ¿Cuándo ocurren más accidentes?**

**¿En qué años ha habido más accidentes?**

In [6]:
# Consulta de accidentes por años
query = """
SELECT
    EXTRACT(YEAR FROM timestamp) AS anio,
    COUNT(*) AS total_accidentes
FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions`
GROUP BY anio
ORDER BY anio;

"""

# Ejecutar la consulta y obtener los datos en un DataFrame
accidents_by_year = ejecutar_consulta(query)

In [7]:
# Crear el gráfico de líneas
fig = px.line(accidents_by_year,
                 x="anio",
                 y="total_accidentes",
                 title="Total de Accidentes de Tráfico por Año",
                 labels={"anio": "Año", "total_accidentes": "Total de Accidentes"},
                 markers=True)

# Ajustar el formato del eje Y
fig.update_layout(yaxis=dict(tickformat=",.0f"))

# Mostrar la figura
fig.show()

**¿Cuáles son las horas con más accidentes?**

In [8]:
query = """
SELECT
    EXTRACT(HOUR FROM timestamp) AS hora,
    COUNT(*) AS total_accidentes
FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions`
GROUP BY hora
ORDER BY hora;
"""
# Ejecutar la consulta y obtener los datos en un DataFrame
accidents_by_hour = ejecutar_consulta(query)


In [9]:
# Crear el gráfico de líneas
fig = px.line(accidents_by_hour,
              x="hora",
              y="total_accidentes",
              title="Total de Accidentes de Tráfico por Hora",
              labels={"hora": "Hora del Día", "total_accidentes": "Total de Accidentes"},
              markers=True)

# Ajustar el formato del eje Y para mostrar números sin decimales ni "K" incorrectos
fig.update_layout(yaxis=dict(tickformat=",.0f"))

# Mostrar la figura
fig.show()


**¿Qué meses del año tienen más accidentes?**

In [10]:
query = """
SELECT
    EXTRACT(MONTH FROM timestamp) AS mes,
    COUNT(*) AS total_accidentes
FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions`
GROUP BY mes
ORDER BY mes;
"""

# Ejecutar la consulta y obtener los datos en un DataFrame
accident_by_month = ejecutar_consulta(query)

In [11]:
# Crear el gráfico de líneas para accidentes por mes
fig = px.line(accident_by_month,
              x="mes",
              y="total_accidentes",
              title="Total de Accidentes de Tráfico por Mes",
              labels={"mes": "Mes", "total_accidentes": "Total de Accidentes"},
              markers=True)

# Ajustar el formato del eje Y para mostrar números sin decimales ni "K" incorrectos
fig.update_layout(yaxis=dict(tickformat=",.0f"))

# Mostrar la figura
fig.show()


### **Análisis Geográfico: ¿Dónde ocurren más accidentes?**

In [12]:
query = """
SELECT
    borough,
    COUNT(*) AS total_accidentes
FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions`
WHERE borough IS NOT NULL
GROUP BY borough
ORDER BY total_accidentes DESC;
"""
# Ejecutar la consulta y obtener los datos en un DataFrame
accident_by_borough = ejecutar_consulta(query)

In [13]:
# Crear el gráfico de barras para accidentes por barrio (borough)
fig = px.bar(accident_by_borough,
             x="borough",
             y="total_accidentes",
             title="Total de Accidentes de Tráfico por Barrio en NYC",
             labels={"borough": "Barrio", "total_accidentes": "Total de Accidentes"},
             color="borough",  # Asignar un color diferente a cada barra
             color_discrete_sequence=px.colors.qualitative.Set1)  # Puedes usar diferentes paletas de colores

# Mostrar la figura
fig.show()


### **¿Cuáles son las calles con más accidentes?**

In [14]:
query = """
SELECT
    on_street_name,
    COUNT(*) AS total_accidentes
FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions`
WHERE on_street_name IS NOT NULL
GROUP BY on_street_name
ORDER BY total_accidentes DESC
LIMIT 10;
"""
# Ejecutar la consulta y obtener los datos en un DataFrame
accident_by_street = ejecutar_consulta(query)

In [15]:
# Crear el gráfico de barras para accidentes por nombre de calle
fig = px.bar(accident_by_street,
             x="on_street_name",
             y="total_accidentes",
             title="Top 10 Accidentes de Tráfico por Nombre de Calle en NYC",
             labels={"on_street_name": "Nombre de la Calle", "total_accidentes": "Total de Accidentes"},
             color="on_street_name",  # Asignar un color diferente a cada barra
             color_discrete_sequence=px.colors.qualitative.Set1)  # Paleta de colores para las barras

# Desactivar la leyenda de colores
fig.update_layout(showlegend=False)

# Ajustar el formato del eje Y para mostrar números sin decimales ni "K" incorrectos
fig.update_layout(yaxis=dict(tickformat=",.0f"))

# Mostrar la figura
fig.show()



### **Causas de los Accidentes: ¿Por qué ocurren?**

**Top 10 causas de accidentes (excluyendo "Unspecified")**

In [16]:
query = """
SELECT
    contributing_factor_vehicle_1 AS factor,
    COUNT(*) AS cantidad_accidentes
FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions`
WHERE contributing_factor_vehicle_1 != 'Unspecified'
GROUP BY factor
ORDER BY cantidad_accidentes DESC
LIMIT 10;

"""
# Ejecutar la consulta y obtener los datos en un DataFrame
accident_by_factor = ejecutar_consulta(query)

# Mostrar el resultado
accident_by_factor

,factor,cantidad_accidentes
0,Driver Inattention/Distraction,435902
1,Failure to Yield Right-of-Way,129003
2,Following Too Closely,115857
3,Backing Unsafely,79230
4,Other Vehicular,67070
5,Passing or Lane Usage Improper,61309
6,Passing Too Closely,54453
7,Turning Improperly,53191
8,Fatigued/Drowsy,47481
9,Unsafe Lane Changing,42469


**¿Cuántos accidentes han ocurrido por alcohol/drogas?**

In [17]:
query = """
SELECT
    COUNT(*) AS accidentes_alcohol_drogas
FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions`
WHERE contributing_factor_vehicle_1 LIKE '%Alcohol%'
   OR contributing_factor_vehicle_1 LIKE '%Drugs%';
"""
# Ejecutar la consulta y obtener los datos en un DataFrame
accident_by_sustance = ejecutar_consulta(query)

# Mostrar el resultado
accident_by_sustance

,accidentes_alcohol_drogas
0,25737


### **Consecuencias de los Accidentes: ¿Cuántos heridos y fallecidos hay?**

In [18]:
query = """
SELECT
    COUNT(*) AS accidentes_fatales
FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions`
WHERE number_of_persons_killed > 0;
"""

# Ejecutar la consulta y obtener los datos en un DataFrame
fatal_accidents = ejecutar_consulta(query)

fatal_accidents

,accidentes_fatales
0,3217


**¿Cuál es el promedio de personas heridas por accidente?**

In [19]:
query = """
SELECT
    AVG(number_of_persons_injured) AS promedio_heridos
FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions`;
"""

# Ejecutar la consulta y obtener los datos en un DataFrame
avg_persons_injured = ejecutar_consulta(query)

avg_persons_injured

,promedio_heridos
0,0.320824


**¿Qué distritos tienen más accidentes con víctimas mortales?**

In [20]:
query = """
SELECT
    borough,
    COUNT(*) AS accidentes_fatales
FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions`
WHERE number_of_persons_killed > 0
GROUP BY borough
ORDER BY accidentes_fatales DESC;
"""

# Ejecutar la consulta y obtener los datos en un DataFrame
borough_fatal_accident = ejecutar_consulta(query)

borough_fatal_accident

,borough,accidentes_fatales
0,None,1285
1,BROOKLYN,657
2,QUEENS,531
3,MANHATTAN,350
4,BRONX,295
5,STATEN ISLAND,99
